## CNN Working Details

In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset,Dataset,DataLoader
from torch.optim import Adam
from torchvision import datasets
from torch.nn import functional as F
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
device='cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
X_train = torch.tensor([[[[1,2,3,4],[2,3,4,5],[5,6,7,8],[1,3,4,5]]],[[[-1,2,3,-4],[2,-3,4,5],[-5,6,-7,8],[-1,-3,-4,-5]]]]).to(device).float()
X_train /= 8
y_train = torch.tensor([0,1]).to(device).float()

In [3]:
def get_model():
    model=nn.Sequential(
        nn.Conv2d(1,1,kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(1,1),
        nn.Sigmoid()
    ).to(device)
    loss_fn=nn.BCELoss()
    optimizer=Adam(model.parameters(),lr=1e-2)
    return model,loss_fn,optimizer
    

In [4]:
model,loss_fn,optimizer=get_model()
summary(model,X_train.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 1, 2, 2]              10
         MaxPool2d-2              [-1, 1, 1, 1]               0
              ReLU-3              [-1, 1, 1, 1]               0
           Flatten-4                    [-1, 1]               0
            Linear-5                    [-1, 1]               2
           Sigmoid-6                    [-1, 1]               0
Total params: 12
Trainable params: 12
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [5]:
def train_batch(x,y,model,opt,loss_fn):
    model.train()
    batch_loss=loss_fn(model(x),y)
    batch_loss.backward()
    opt.step()
    opt.zero_grad()
    return batch_loss.item()

In [6]:
trn_dl=DataLoader(TensorDataset(X_train,y_train))

In [7]:
for epoch in range(2000):
    for ix,batch in enumerate(iter(trn_dl)):
        x,y=batch
        batch_loss=train_batch(x,y.reshape(1,1),model,optimizer,loss_fn)

In [8]:
model(X_train[:1])

tensor([[0.5001]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [9]:
list(model.children())

[Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 ReLU(),
 Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=1, out_features=1, bias=True),
 Sigmoid()]

In [10]:
(cnn_w,cnn_b),(lin_w,lin_b)=[(layer.weight.data,layer.bias.data) for layer in list(model.children()) if hasattr(layer,'weight')]

In [11]:
h_im,w_im=X_train.shape[2:]
h_conv,w_conv=cnn_w.shape[2:]
sumprob=torch.zeros((h_im-h_conv+1),(w_im-w_conv+1))

In [12]:
for i in range(h_im-h_conv+1):
    for j in range(w_im-w_conv+1):
        img_subset=X_train[0,0,i:i+3,j:j+3]
        model_filter=cnn_w.reshape(3,3)
        val=torch.sum(img_subset*model_filter)+cnn_b
        sumprob[i,j]=val

In [13]:
sumprob.clamp_min(0)

tensor([[0., 0.],
        [0., 0.]])

In [14]:
pooling_layer_output=torch.max(sumprob)

In [15]:
intermediate_output_layer=pooling_layer_output*lin_w+lin_b

In [16]:
print(F.sigmoid(intermediate_output_layer))

tensor([[0.6288]], device='cuda:0')
